<a href="https://colab.research.google.com/github/mrcrchln/Machine-Learning/blob/main/2487_Final_Exam_2223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2487-2223 Machine Learning S2 Final Exam

The final exam will contain 1 question with subquestions for 100% of the total points.


## Animal rights are human rights (100 points)



![image](https://assets.change.org/photos/9/rj/ub/tSRjUbxDZFlzLaj-800x450-noPad.jpg)

Animal rights are moral principles grounded in the belief that non-human animals deserve the ability to live as they wish, without being subjected to the desires of human beings. At the core of animal rights is autonomy, which is another way of saying choice. In many countries, human rights are enshrined to protect certain freedoms, such as the right to expression, freedom from torture, and access to democracy. It is becoming increasingly difficult to deny the ethical and practical implications of animal welfare. Animal rights aim to do something similar, only for non-human animals.

The same applies to the pet animals. The European Convention for the Protection of Pet Animals (ETS No. 125) contains provisions to protect pet animals, and seeks to establish a basic common standard of attitude and practice towards pet ownership. Provisions are included on breeding, boarding, keeping of pet animals. The Convention also aims at regulating trading in and breeding of pet animals, at prohibiting the modification of their natural appearance and at reducing the number of stray animals.




**Petfinder data (petfinder-mini.csv)**
Petfinder is an internet company that operates the largest online pet adoption website serving all of North America. The company reports that it currently lists “more than 315,000 adoptable pets from nearly 14,000 animal shelters and rescue groups." Most of the pets listed on Petfinder are dogs and cats, but they list all types of animals available from shelters and rescue groups, from small fish, reptiles and birds to horses and livestock

There are several thousand rows in the Petfinder's CSV dataset file, where each row describes a pet (a dog or a cat) and each column describes an attribute, such as age, breed, color, and so on.

Below you may find the column summary

| Column          | Pet description               | Feature type   | Data type |
| --------------- | ----------------------------- | -------------- | --------- |
| `Type`          | Type of animal (`Dog`, `Cat`) | Categorical    | String    |
| `Age`           | Age                           | Numerical      | Integer   |
| `Breed1`        | Primary breed                 | Categorical    | String    |
| `Color1`        | Color 1                       | Categorical    | String    |
| `Color2`        | Color 2                       | Categorical    | String    |
| `MaturitySize`  | Size at maturity              | Categorical    | String    |
| `FurLength`     | Fur length                    | Categorical    | String    |
| `Vaccinated`    | Pet has been vaccinated       | Categorical    | String    |
| `Sterilized`    | Pet has been sterilized       | Categorical    | String    |
| `Health`        | Health condition              | Categorical    | String    |
| `Fee`           | Adoption fee                  | Numerical      | Integer   |
| `Description`   | Profile write-up              | Text           | String    |
| `PhotoAmt`      | Total uploaded photos         | Numerical      | Integer   |
| `AdoptionSpeed` | Categorical speed of adoption | Classification | Integer   |

Answer the following questions using the provided dataset. You can write down intermediate results towards the final answers. If any model invovles `random_state`, set it to be 42.

In [15]:
# Import Libraries
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')

In [2]:
# conect google drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset_path = '/content/drive/My Drive/T4/EXAM/final_exam_2223/petfinder-mini.csv'

petfinder = pd.read_csv(dataset_path)
petfinder.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


The target variable is `AdoptionSpeed`: Number of weeks animal was in the shelter until they were adopted.

In [5]:
petfinder['AdoptionSpeed'].value_counts()

2    3153
4    3080
3    2543
1    2432
0     329
Name: AdoptionSpeed, dtype: int64

In [6]:
petfinder.describe()

,Age,Fee,PhotoAmt,AdoptionSpeed
count,11537.000000,11537.000000,11537.000000,11537.000000
mean,11.743434,23.957268,3.610211,2.486522
std,19.324221,80.024226,3.145872,1.173275
min,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,2.000000,2.000000
50%,4.000000,0.000000,3.000000,2.000000
75%,12.000000,0.000000,5.000000,4.000000
max,255.000000,2000.000000,30.000000,4.000000


In [7]:
X = petfinder.drop(columns=['AdoptionSpeed', 'Description'])
y = petfinder['AdoptionSpeed']

### Question 1 (20 points)

Split the data into 80% training set and 20% test set. Build a pipeline to transform the features. For categorical features, transform them through one-hot encoding. For numerical features, use robust scaling.


Train Lositic Regression Models (with 100 maximum iterations and saga solver) with L1 regularization using 5-fold cross validation. You could choose 1) One-vesus-Rest (OvR) or 2) One-versus-One (OvO) or 3) Multinomial strategies and compare the total training time and average cross-validationg scores (in terms of marco-average precision). Explain your findings.

In [8]:
# Display all columns and their data types
print(petfinder.dtypes)

Type             object
Age               int64
Breed1           object
Gender           object
Color1           object
Color2           object
MaturitySize     object
FurLength        object
Vaccinated       object
Sterilized       object
Health           object
Fee               int64
Description      object
PhotoAmt          int64
AdoptionSpeed     int64
dtype: object


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler

# Split the data into features (X) and the target variable (y)
X = petfinder.drop('AdoptionSpeed', axis=1)
y = petfinder['AdoptionSpeed']

# Split the data into 80% training set and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer for feature transformation
categorical_cols = ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health']
numerical_cols = ['Age', 'Fee', 'PhotoAmt']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', RobustScaler(), numerical_cols)
    ])

# Build the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the training data
X_train_transformed = pipeline.fit_transform(X_train)

# Transform the test data
X_test_transformed = pipeline.transform(X_test)

In [10]:
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score
from sklearn.multiclass import OneVsOneClassifier

# Set the parameters
max_iter = 100
solver = 'saga'

# Create the Logistic Regression models with different strategies
ovr_model = LogisticRegression(penalty='l1', solver=solver, multi_class='ovr', max_iter=max_iter, class_weight='balanced')
ovo_model = OneVsOneClassifier(LogisticRegression(penalty='l1', solver=solver, max_iter=max_iter, class_weight='balanced'))
multinomial_model = LogisticRegression(penalty='l1', solver=solver, multi_class='multinomial', max_iter=max_iter, class_weight='balanced')

# Define the macro-average precision scorer for cross-validation
scorer = make_scorer(precision_score, average='macro')

# Perform cross-validation and measure training time
start_time = time.time()
ovr_scores = cross_val_score(ovr_model, X_train_transformed, y_train, cv=5, scoring=scorer)
ovr_training_time = time.time() - start_time

start_time = time.time()
ovo_scores = cross_val_score(ovo_model, X_train_transformed, y_train, cv=5, scoring=scorer)
ovo_training_time = time.time() - start_time

start_time = time.time()
multinomial_scores = cross_val_score(multinomial_model, X_train_transformed, y_train, cv=5, scoring=scorer)
multinomial_training_time = time.time() - start_time

# Print the results
print("One-versus-Rest (OvR):")
print("Training Time:", ovr_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(ovr_scores))

print("\nOne-versus-One (OvO):")
print("Training Time:", ovo_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(ovo_scores))

print("\nMultinomial:")
print("Training Time:", multinomial_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(multinomial_scores))

One-versus-Rest (OvR):
Training Time: 20.813547611236572
Average Cross-Validation Score (Macro-average Precision): 0.27968330683249853

One-versus-One (OvO):
Training Time: 16.130016326904297
Average Cross-Validation Score (Macro-average Precision): 0.27903135905916365

Multinomial:
Training Time: 21.532047748565674
Average Cross-Validation Score (Macro-average Precision): 0.2816409734065982


One-versus-Rest (OvR): The OvR strategy took the longest training time (around 21.46 seconds) among the three strategies. It achieved an average macro-average precision of approximately 0.279. In the OvR approach, one classifier is trained for each class against all other classes. While it may have longer training time, it can handle multi-class classification efficiently.

One-versus-One (OvO): The OvO strategy was faster than the OvR strategy, taking about 13.54 seconds to train. It achieved a similar macro-average precision of approximately 0.279. In the OvO approach, a classifier is trained for each pair of classes. It can sometimes lead to better performance by considering pairwise decision boundaries.

Multinomial: The multinomial strategy had a training time similar to the OvO strategy, around 13.72 seconds. It achieved a slightly higher macro-average precision of approximately 0.281. The multinomial approach treats the multi-class problem as a single optimization problem, considering all classes simultaneously. It can be computationally efficient and provides a more direct approach to multi-class classification.

Multinomial seems to be the clear choice.

### Question 2 (20 points)

Now looking at the coefficients from Logitstic Regression with multinomial strategy from the previous question, train this model with all features and identify coefficients. You may find some features with the esitmated coefficients to be 0. Cross-validate the same model using only features that with non-zero coefficients. Note that you may have five set of coefficents for five classes and you can filter out the features with coefficients that are zeros for all classes. Compare your model performance using 5-fold cross validation and report time used for training as well as the test score. Explain your findings by comparing it with the previous result.




In [11]:
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score

# Create the Logistic Regression model with multinomial strategy
multinomial_model = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=100, class_weight='balanced')

# Fit the model on the entire training data
multinomial_model.fit(X_train_transformed, y_train)

# Get the coefficients for each class
coefficients = multinomial_model.coef_

# Filter out features with coefficients that are zero for all classes
nonzero_indices = np.any(coefficients != 0, axis=0)
X_train_transformed_filtered = X_train_transformed[:, nonzero_indices]
X_test_transformed_filtered = X_test_transformed[:, nonzero_indices]

# Perform cross-validation and measure training time
start_time = time.time()
filtered_scores = cross_val_score(multinomial_model, X_train_transformed_filtered, y_train, cv=5, scoring=scorer)
filtered_training_time = time.time() - start_time

# Fit the model to the filtered training data and calculate the test score
multinomial_model.fit(X_train_transformed_filtered, y_train)
test_score = multinomial_model.score(X_test_transformed_filtered, y_test)

# Print the results
print("Filtered Features (Non-zero Coefficients) - Multinomial:")
print("Training Time:", filtered_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(filtered_scores))
print("Test Score:", test_score)

Filtered Features (Non-zero Coefficients) - Multinomial:
Training Time: 15.684412956237793
Average Cross-Validation Score (Macro-average Precision): 0.281590325600195
Test Score: 0.3210571923743501


In the previous question, we trained a Logistic Regression model using the Multinomial strategy, considering all the features. Now, we trained the same model but only include the features with non-zero coefficients. This allows us to filter out irrelevant features and potentially improve the model's performance.

After filtering the features based on non-zero coefficients, we obtained the following results:


Filtered Features (Non-zero Coefficients) - Multinomial:

Training Time: 14.241383790969849

Average Cross-Validation Score (Macro-average Precision): 0.28272237278163737

Test Score: 0.3210571923743501


Comparing these results with the previous findings, we can observe the following:

Training Time: The training time for the model using filtered features with non-zero coefficients is slightly higher than the previous model that used all features (around 14.24 seconds compared to 13.72 seconds). This suggests that the filtering step potentially increased computational power.

Cross-Validation Score: The average macro-average precision for the filtered features model is 0.283, which is slightly higher than the previous model's score of 0.281. This indicates a slight improvement in performance when using only the non-zero coefficient features.

Test Score: The test score for the filtered features model is 0.321. Which is rather low.

Overall, the model with filtered features and non-zero coefficients showed a slightly improved cross-validation score compared to the previous model that used all features. And training time slightly increased.

### Question 3 (20 points)

Alternatively, you could perform the PCA and reduce the dimension of the transformed X into the same dimension as the previous question. Add the pca step and the Logisticregression (with the same setup: with 100 maximum iterations, saga solver, L1 regularization into the pipeline). Cross-validate the pipeline on X_train, y_train and report your mean test scores and total training time. Explain your findings by comparing with the previous results.




Note: TruncatedSVD is used here instead of PCA because PCA cannot be applied to sparse data, which is common in text and other high-dimensional problems. TruncatedSVD performs linear dimensionality reduction, unlike PCA, it does not require the input data to be centered and works with sparse input data.

In [12]:
from sklearn.decomposition import TruncatedSVD

# Define the TruncatedSVD and Logistic Regression steps
svd = TruncatedSVD(n_components=X_train_transformed_filtered.shape[1])
logreg = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=100, class_weight='balanced')

# Create a pipeline with TruncatedSVD and Logistic Regression
pipeline = Pipeline(steps=[('svd', svd), ('logistic', logreg)])

# Perform cross-validation and measure training time
start_time = time.time()
pipeline_scores = cross_val_score(pipeline, X_train_transformed, y_train, cv=5, scoring=scorer)
pipeline_training_time = time.time() - start_time

# Print the results
print("TruncatedSVD + Logistic Regression:")
print("Training Time:", pipeline_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(pipeline_scores))

TruncatedSVD + Logistic Regression:
Training Time: 39.6456880569458
Average Cross-Validation Score (Macro-average Precision): 0.2815841248127124


In this exercise, we used TruncatedSVD for dimensionality reduction instead of PCA because TruncatedSVD is better suited for sparse data. We added the TruncatedSVD and Logistic Regression steps into a pipeline, with L1 regularization, saga solver, and 100 maximum iterations.

The results of applying TruncatedSVD followed by Logistic Regression are as follows:

TruncatedSVD + Logistic Regression:
- Training Time: The total training time for the pipeline is approximately 33.88 seconds. This includes both the TruncatedSVD dimensionality reduction step and the training of the logistic regression model.
- Average Cross-Validation Score (Macro-average Precision): The average macro-average precision obtained during cross-validation is around 0.28.

Comparing these findings with the previous results, we can observe the following:

Training Time: The training time for the TruncatedSVD + Logistic Regression pipeline is longer compared to both the Multinomial Logistic Regression model without feature reduction and the Multinomial Logistic Regression model with filtered features. This is expected as dimensionality reduction with TruncatedSVD involves additional computations.

Average Cross-Validation Score: The average cross-validation score using the TruncatedSVD + Logistic Regression pipeline is very similar to the previous results. However, it is still lower compared to the Multinomial Logistic Regression model without feature reduction. This indicates that the dimensionality reduction performed by TruncatedSVD may have resulted in some loss of information, leading to a slightly reduced performance in terms of macro-average precision.

Based on these findings, it appears that the TruncatedSVD + Logistic Regression approach did not provide significant improvements compared to the previous methods. The longer training time and similar or slightly lower performance suggest that the dimensionality reduction with TruncatedSVD did not effectively capture the essential information for the classification task in this scenario.

### Question 4 (20 points)

Now let's try to engineer some new features by creating the cluster labels for each pet. You need to perform K-means algorithm to cluster pet according to their characteristics and add the cluster labels as the additional feature. You could perform K-means on the pca transformed feature (i.e. K-means step appear after the PCA). Once you get the cluster labels, add them as the one-hot-encoded features to your PCA transformed features. Run the same LogisticRegression and see if the performance improves. You can set n_clusters to be 10.



In [13]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD

# Define TruncatedSVD and KMeans steps
svd = TruncatedSVD(n_components=X_train_transformed_filtered.shape[1])
kmeans = KMeans(n_clusters=10, random_state=42)

# Perform TruncatedSVD and KMeans clustering
X_train_transformed_svd = svd.fit_transform(X_train_transformed)
kmeans.fit(X_train_transformed_svd)

# Get cluster labels and one-hot encode them
cluster_labels = kmeans.labels_
encoder = OneHotEncoder(sparse=False)
cluster_labels_encoded = encoder.fit_transform(cluster_labels.reshape(-1, 1))

# Add cluster labels to TruncatedSVD-transformed features
X_train_transformed_svd_clustered = np.hstack((X_train_transformed_svd, cluster_labels_encoded))

# Create and fit logistic regression model
logreg = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=100, class_weight='balanced')

# Perform cross-validation and measure training time
start_time = time.time()
clustered_scores = cross_val_score(logreg, X_train_transformed_svd_clustered, y_train, cv=5, scoring=scorer)
clustered_training_time = time.time() - start_time

# Print the results
print("TruncatedSVD + KMeans + Logistic Regression:")
print("Training Time:", clustered_training_time)
print("Average Cross-Validation Score (Macro-average Precision):", np.mean(clustered_scores))

TruncatedSVD + KMeans + Logistic Regression:
Training Time: 34.87935018539429
Average Cross-Validation Score (Macro-average Precision): 0.2835211289481639


The addition of cluster labels as features using the TruncatedSVD + KMeans + Logistic Regression pipeline did not really improve the model's performance Average Cross-Validation Score (Macro-average Precision): 0.2827847419182778 is basically the same while training time increased.

### Question 5 (20 points)

Given that ithe original dataset adoption speed of "4" indicates the pet was not adopted, let's convert this problem into predicting a pet will be adopted or not. You can create the label and check whether the cat and dog would have the same probablity to be adopted. If we use the same model to retrain on the X_train and predict on the X_test, show the classification report. Explain whether the model is able to generate similar predictions for cat and dog.

In [16]:
# Convert the problem into a binary classification problem (adopted or not adopted)
y_train_binary = (y_train != 4).astype(int)
y_test_binary = (y_test != 4).astype(int)

# Create and fit logistic regression model with balanced class weights
logreg = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=100, class_weight='balanced')

# Fit and predict with the logistic regression model
logreg.fit(X_train_transformed, y_train_binary)
y_test_pred = logreg.predict(X_test_transformed)

# Create masks for cats and dogs
mask_cats = (X_test['Type'] == 'Cat')
mask_dogs = (X_test['Type'] == 'Dog')

# Separate the test set and predictions into cats and dogs
y_test_cats = y_test_binary[mask_cats]
y_test_dogs = y_test_binary[mask_dogs]

y_test_pred_cats = y_test_pred[mask_cats]
y_test_pred_dogs = y_test_pred[mask_dogs]

# Generate a classification report for cats
report_cats = classification_report(y_test_cats, y_test_pred_cats, target_names=['Not Adopted', 'Adopted'])
print("Classification Report - Cats:")
print(report_cats)

# Generate a classification report for dogs
report_dogs = classification_report(y_test_dogs, y_test_pred_dogs, target_names=['Not Adopted', 'Adopted'])
print("Classification Report - Dogs:")
print(report_dogs)

Classification Report - Cats:
              precision    recall  f1-score   support

 Not Adopted       0.44      0.44      0.44       256
     Adopted       0.81      0.81      0.81       738

    accuracy                           0.71       994
   macro avg       0.62      0.62      0.62       994
weighted avg       0.71      0.71      0.71       994

Classification Report - Dogs:
              precision    recall  f1-score   support

 Not Adopted       0.41      0.72      0.52       387
     Adopted       0.83      0.56      0.67       927

    accuracy                           0.61      1314
   macro avg       0.62      0.64      0.60      1314
weighted avg       0.71      0.61      0.63      1314



Cats:

The model has an accuracy of 71%, which means it correctly predicted whether a cat was adopted or not 71% of the time.
For predicting adoption (the "Adopted" class), the model has a precision of 0.81, which means that 81% of the cats that were predicted to be adopted were actually adopted.
The recall of 0.81 for the "Adopted" class means that the model correctly identified 81% of all adopted cats.
For predicting not adopted (the "Not Adopted" class), the model has a precision and recall of 0.44. This means that the model's performance in predicting not adopted cats is relatively poor compared to predicting adopted cats.

Dogs:

The model has an accuracy of 61%, which is lower than that for cats.
For predicting adoption (the "Adopted" class), the model has a precision of 0.83, but the recall is significantly lower at 0.56. This suggests that while the model is fairly reliable when it predicts a dog will be adopted (83% of dogs predicted to be adopted were actually adopted), it is less successful in identifying all the dogs that were adopted (it only correctly identified 56% of all adopted dogs).
For predicting not adopted (the "Not Adopted" class), the model has a precision of 0.41 but a relatively high recall of 0.72. This means that while only 41% of dogs predicted to be not adopted were actually not adopted, the model was able to correctly identify 72% of all not adopted dogs.


Overall, the model seems to be more accurate for cats than for dogs. For both animals, the model performs better in predicting adoption than predicting non-adoption. However, the difference is more pronounced for dogs. The model is particularly weak at predicting dogs that were not adopted, as indicated by the low precision for the "Not Adopted" class.

Congratulations! Now you are a qualified business analyst.